In [26]:
import psycopg2

/Users/aryanpatel/.pyenv/versions/3.9.16/envs/python-3.9.16-virtual/lib/python3.9/site-packages/psycopg2/__init__.py:51: RuntimeWarning: coroutine 'main' was never awaited
  from psycopg2._psycopg import (                     # noqa
/Users/aryanpatel/.pyenv/versions/3.9.16/envs/python-3.9.16-virtual/lib/python3.9/site-packages/urllib3/util/connection.py:7: RuntimeWarning: coroutine 'main' was never awaited
  from .timeout import _DEFAULT_TIMEOUT, _TYPE_TIMEOUT


In [27]:
conn=psycopg2.connect(database="stockgraphdb",
                 user="aryanpatel",
                 password="12345",
                 host="localhost",
                 port="5432")

cursor=conn.cursor()

In [28]:
sqlQuery="""INSERT INTO adanient.adanient_1m (open,close,high,low,volume,epochtime) VALUES(2.00, 4.0, 3.5 ,4.5 ,2244 ,1682513967)
ON CONFLICT(epochtime)  
DO UPDATE SET
open = EXCLUDED.open,
close=EXCLUDED.close,
high=EXCLUDED.high,
volume=EXCLUDED.volume;
"""


try:
    cursor.execute(sqlQuery)
    conn.commit()
except Exception as e:
    print("Error:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

Error: relation "adanient.adanient_1m" does not exist
LINE 1: INSERT INTO adanient.adanient_1m (open,close,high,low,volume...
                    ^



In [29]:
import pytz
from datetime import datetime, timedelta
def generate_groww_api_url(symbol, start_date_str, end_date_str, interval_minutes):
    ist = pytz.timezone("Asia/Kolkata")
    
    # Convert string dates to datetime
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # Set time to 9:15 AM for start date
    start_datetime = ist.localize(datetime.combine(start_date, datetime.strptime("09:15", "%H:%M").time()))
    
    # Set time to 3:30 PM for end date
    end_datetime = ist.localize(datetime.combine(end_date, datetime.strptime("15:30", "%H:%M").time()))
    
    # Convert to milliseconds
    start_millis = int(start_datetime.timestamp() * 1000)
    end_millis = int(end_datetime.timestamp() * 1000)
    
    url = f"https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/{symbol}?intervalInMinutes={interval_minutes}&startTimeInMillis={start_millis}&endTimeInMillis={end_millis}"
    
    return url

# Example usage:

symbol = "ADANIENT"
start_date = "2024-05-08"   # 8th May 2024
end_date = "2025-04-25"     # 25th April 2025
interval = 1                # 1 minute

url = generate_groww_api_url(symbol, start_date, end_date, interval)
print(url)

https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/ADANIENT?intervalInMinutes=1&startTimeInMillis=1715139900000&endTimeInMillis=1745575200000


In [30]:
def generate_intraday_intervals(end_date, interval_name):
    ist = pytz.timezone("Asia/Kolkata")

    # Fixed start date: Wednesday, 8 May 2024, 09:15 AM IST
    fixed_start_date = datetime(2024, 5, 8, 9, 15)
    fixed_start_datetime = ist.localize(fixed_start_date)

    # Move end date 7 days back
    end_date_adjusted = end_date - timedelta(days=7)

    # Ensure only weekdays and proper market timing
    intervals = []

    current_date = fixed_start_datetime.date()
    final_date = end_date_adjusted.date()

    interval_minutes = interval_map.get(interval_name, 1)  # default 1 min if not found

    while current_date <= final_date:
        # Only weekdays (Mon-Fri)
        if current_date.weekday() < 5:
            start_datetime = ist.localize(datetime.combine(current_date, datetime.strptime("09:15", "%H:%M").time()))
            end_datetime = ist.localize(datetime.combine(current_date, datetime.strptime("15:30", "%H:%M").time()))

            intervals.append((start_datetime, end_datetime))

        current_date += timedelta(days=1)

    return intervals





In [31]:
start_date = datetime(2025 , 1, 1) # Example start date    
#create intervals every week after 1 week and end week is today

today = datetime.now()
##end date should be startdate + 7 days until today
intervals = []

while start_date < today:
    intervals.append((start_date, start_date + timedelta(days=7)))
    start_date = start_date + timedelta(days=7)

In [32]:
intervals

[(datetime.datetime(2025, 1, 1, 0, 0), datetime.datetime(2025, 1, 8, 0, 0)),
 (datetime.datetime(2025, 1, 8, 0, 0), datetime.datetime(2025, 1, 15, 0, 0)),
 (datetime.datetime(2025, 1, 15, 0, 0), datetime.datetime(2025, 1, 22, 0, 0)),
 (datetime.datetime(2025, 1, 22, 0, 0), datetime.datetime(2025, 1, 29, 0, 0)),
 (datetime.datetime(2025, 1, 29, 0, 0), datetime.datetime(2025, 2, 5, 0, 0)),
 (datetime.datetime(2025, 2, 5, 0, 0), datetime.datetime(2025, 2, 12, 0, 0)),
 (datetime.datetime(2025, 2, 12, 0, 0), datetime.datetime(2025, 2, 19, 0, 0)),
 (datetime.datetime(2025, 2, 19, 0, 0), datetime.datetime(2025, 2, 26, 0, 0)),
 (datetime.datetime(2025, 2, 26, 0, 0), datetime.datetime(2025, 3, 5, 0, 0)),
 (datetime.datetime(2025, 3, 5, 0, 0), datetime.datetime(2025, 3, 12, 0, 0)),
 (datetime.datetime(2025, 3, 12, 0, 0), datetime.datetime(2025, 3, 19, 0, 0)),
 (datetime.datetime(2025, 3, 19, 0, 0), datetime.datetime(2025, 3, 26, 0, 0)),
 (datetime.datetime(2025, 3, 26, 0, 0), datetime.datetime(2

In [33]:
#convert intervasl to epochime
def convert_to_epoch(intervals):
    epoch_intervals = []
    for start, end in intervals:
        start_epoch = int(start.timestamp())
        end_epoch = int(end.timestamp())
        epoch_intervals.append((start_epoch, end_epoch))
    return epoch_intervals

In [34]:
convert_to_epoch(intervals)

[(1735669800, 1736274600),
 (1736274600, 1736879400),
 (1736879400, 1737484200),
 (1737484200, 1738089000),
 (1738089000, 1738693800),
 (1738693800, 1739298600),
 (1739298600, 1739903400),
 (1739903400, 1740508200),
 (1740508200, 1741113000),
 (1741113000, 1741717800),
 (1741717800, 1742322600),
 (1742322600, 1742927400),
 (1742927400, 1743532200),
 (1743532200, 1744137000),
 (1744137000, 1744741800),
 (1744741800, 1745346600),
 (1745346600, 1745951400)]

In [35]:
def connection():
    conn = psycopg2.connect(
        database="stockanalysis",
        user='aryanpatel',
        password="12345",
        host="localhost",
        port="5432"
    )
    return conn

def create_table(conn, stock_name, interval_name):
    
    conn.cursor().execute(f'CREATE SCHEMA IF NOT EXISTS {stock_name}')  # Use quotes to handle special characters

    conn.cursor().execute(f"""
        CREATE TABLE IF NOT EXISTS {stock_name}.candle_{interval_name} (
            id SERIAL PRIMARY KEY,
            open FLOAT,
            high FLOAT,
            low FLOAT,
            close FLOAT,
            volume FLOAT,
            epochtime BIGINT UNIQUE NOT NULL
        )
    """)
    conn.commit()
    

In [36]:
conn = connection()
stock_name = "M&M"
interval_name = "1m"
create_table(conn, stock_name, interval_name)
conn.close()

SyntaxError: syntax error at or near "&"
LINE 1: CREATE SCHEMA IF NOT EXISTS M&M
                                     ^


In [147]:
import asyncio


24


In [47]:
import asyncio

async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

asyncio.run(main()) 
await asyncio.sleep(1, result='hello')


Hello ...
... World!


'hello'

In [51]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

started at 10:07:54
hello
world
finished at 10:07:57


In [73]:
async def main():
    task1 = asyncio.create_task(
        say_after(4, 'hello'))

    task2 = asyncio.create_task(
        say_after(10, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await asyncio.gather(task1,task2)

    print(f"finished at {time.strftime('%X')}")

await main()
 

started at 11:00:31
hello
world
finished at 11:00:41


In [76]:
s1= "test"
s2= lambda func:func.upper()
print(s2(s1))

TEST


In [80]:
def test():
    #local variable
    x="abc"
    print(x)

test()
x="er"
#global vatriable
print(x)

abc
er


In [87]:
a = 1

# Uses global because there is no local 'a'
def f():
    print('Inside f() : ', a)

# Variable 'a' is redefined as a local
def g():
    a = 2
    print('Inside g() : ', a)

# Uses global keyword to modify global 'a'
def h():
    global a
    a = 3
    print('Inside h() : ', a)

print('global : ', a)
f()
print('global : ', a)
g()
print('global : ', a)
h() 
print('global : ', a)


global :  1
Inside f() :  1
global :  1
Inside g() :  2
global :  1
Inside h() :  3
global :  3


In [99]:
def test():
    #local variable
    global x
    x="geeks"
    print(x)

#global variable 
x="test"
test()
print(x)

geeks
geeks


In [101]:
#recursion
def  factorial(n):
    if n==0:
        return 1
    else:
        return n*factorial(n-1)

factorial(5)


120

In [24]:
def fun(*args):
    print(args)

fun(1,2,3,4,5,6)

(1, 2, 3, 4, 5, 6)


In [107]:
def fun(a=None,b=None,c=None):
    print(a,b,c)

fun(a=1,b=2,c=3)



1 2 3


In [108]:
def fun (**kwargs):
    print(kwargs)

fun (a=1,b=2,c=3)


{'a': 1, 'b': 2, 'c': 3}


In [124]:
#use of  self

class Car:
    def __init__(self,model,color):
        self.model=model
        self.color=color
        
    
    def display(self):
        self.model="dg"
    
car1=Car("ABC","red")
car2=Car("ABC","black")
car1.display()
print(car1.model)
print(car2.model)

        
    

dg
ABC


In [25]:
#first class funtion
#passing fun to a variable
def msg(name):
    return f"hello, {name}"

f=msg
print(f("Aryan"))

hello, Aryan


In [131]:
#passing funtion as an arguments
def msg(name):
    return f"hello,{name}"

def fun1(fun2,name):
    return fun2(name)

print(fun1(msg,"Aryan"))

hello,Aryan


In [134]:
#returning funtion from other funtion
def fun1(msg):
    def  fun2():
        return f"hello,{msg}"
    return fun2
func=fun1("Alish")
print(func())

hello,Alish


In [144]:
#filter
n=[1,2,3,4,5]
even=filter(lambda x :x%2==0,n)
print(list(even))

In [148]:
#map()
n=[1,2,3,4,5]
b=map(lambda x :x*2,n)
print(list(b))

[2, 4, 6, 8, 10]


In [3]:
from functools import reduce
a = [1, 2, 3, 4]
b = reduce(lambda x, y: x * y, a)
print(b)  

24


In [11]:
def fun1():#outer funtion
    msg="qwerty"
    def fun2():#inner funtion
        nonlocal msg
        msg="hello"
        print(msg)
    fun2()
    print(msg)
fun1()


hello
hello


In [13]:
def fun1(a): # outer function
    
    def fun2(): # inner function
        print(a)
    return fun2  # returning function without parentheses

closure_func = fun1("Hello, Closure!")
closure_func()  # inner function remembers 'a'

Hello, Closure!


In [15]:
# A simple decorator function
##what is happeing here is that the decorator function takes a function as an argument and returns a new function that adds some functionality before and after calling the original function.
# Decorator function
def decorator(func):
  
    def wrapper():
        print("Before calling the function.")
        func()
        print("After calling the function.")
    return wrapper

# Applying the decorator to a function
@decorator

def greet():
    print("Hello, World!")

greet()


Before calling the function.
Hello, World!
After calling the function.


In [19]:
@decorator
def add(a,b):
    print(a+b)

def decorator(func):
    def wrapper(a,b):
        print("before the funtion")
        func(a,b)
        print("after function")
    return wrapper

add(2,4)

before the funtion
6
after function


In [26]:
#custom decorator
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello")

say_hello()

Something is happening before the function is called.
Hello
Something is happening after the function is called.


In [30]:
##decorators for logging
def log(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function '{func.__name__}' with arguments {args} and keyword arguments {kwargs}")
        result = func(*args, **kwargs)
        print(f"Function '{func.__name__}' returned {result}")
        return result
    return wrapper
@log
def add(a, b):
    return a + b

add(2, 3)

Calling function 'add' with arguments (2, 3) and keyword arguments {}
Function 'add' returned 5


5

In [31]:
##diffrent decorator
def decorator(func):
    def wrapper(*args, **kwargs):
        print("Before the function is called.")
        result = func(*args, **kwargs)
        print("After the function is called.")
        return result
    return wrapper
@decorator
def my_function(x, y):
    return x + y
result = my_function(5, 3)
print(result)  # Output: 8
#decorator with arguments
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat
@repeat(num_times=3)
def greet(name):
    print(f"Hello, {name}!")
greet("Alice")  # Output: Hello, Alice! (printed 3 times)


Before the function is called.
After the function is called.
8
Hello, Alice!
Hello, Alice!
Hello, Alice!


In [34]:
s = "hello geeks"

# Updating by creating a new string
s1 = "H" + s[1:]

# replacnig "geeks" with "GeeksforGeeks"
s2 = s.replace("geeks", "GeeksforGeeks")
print(s1)
print(s2)


Hello geeks
hello GeeksforGeeks


In [36]:
##copy
import copy
# Original list
original_list = [1, 2, 3, [4, 5]]
# Shallow copy
shallow_copied_list = copy.copy(original_list)
# Deep copy
deep_copied_list = copy.deepcopy(original_list)
# Modifying the original list
original_list[3][0] = 'changed'
# Printing the lists
print("Original List:", original_list)  # Original list is modified
print("Shallow Copied List:", shallow_copied_list)  # Shallow copy is affected
print("Deep Copied List:", deep_copied_list)  # Deep copy remains unchanged


Original List: [1, 2, 3, ['changed', 5]]
Shallow Copied List: [1, 2, 3, ['changed', 5]]
Deep Copied List: [1, 2, 3, [4, 5]]


In [37]:
def fun(max):
    cnt = 1
    while cnt <= max:
        yield cnt
        cnt += 1

ctr = fun(5)
for n in ctr:
    print(n)


1
2
3
4
5


In [39]:
def missig_number(arr):
    n=len(arr)+1
    sum_of_arr=sum(arr)
    actual_sum=n*(n+1)//2
    return actual_sum-sum_of_arr

missig_number([1, 2, 3, 5])

4

In [41]:
def getSecondLargest( arr):
    n=len(arr)
    first=float('-inf')
    second=float('-inf')
    for i in range(n):
        if arr[i]>first:
            second=first
            first=arr[i]
        elif arr[i]<first and arr[i]>second:
            second=arr[i]
    return second

getSecondLargest([1, 2, 3, 5, 4])


4

In [46]:
def leaders():
    arr=[16, 17, 4, 3, 5, 2]
    n=len(arr)
    leaders_list=[]
    max_from_right=arr[-1]
    leaders_list.append(max_from_right)
    for  i in range(n-2,-1,-1):
        if arr[i]>max_from_right:
            max_from_right=arr[i]
            leaders_list.append(max_from_right)
    return leaders_list[::-1]

print(leaders())     
       
             

[17, 5, 2]


In [48]:
def arr_duplicate():
    arr=(2, 3, 1, 2, 3)
    n=len(arr)
    duplicates=[]
    freq_map={}
    for i in arr:
        if i in freq_map:
            freq_map[i]+=1
        else:
            freq_map[i]=1
    for key, value in freq_map.items():
        if value>1:
            duplicates.append(key)
    return duplicates

print(arr_duplicate())

        
    
    

[2, 3]


In [50]:
#missing and repeating number
def missing_repeating_number():
    arr=[1, 3, 3] 
    n=len(arr)
    repeating=-1
    actual_sum=0
    seen=set()
    for i in arr:
        if i in seen:
            seen.add(i)
            repeating=i
        else:
            seen.add(i)
            actual_sum+=i
    expected_sum=n*(n+1)//2
    missinng_sum=expected_sum-actual_sum
    return [missinng_sum, repeating]

print(missing_repeating_number())

[2, 3]


In [1]:
def findTwoElement( ): 
        # code here
    arr=[2 ,2]
    n=len(arr)
    repeating=-1
    actual_sum=0
    seen=set()
    for i in arr:
        if i in seen:
            seen.add(i)
            repeating=i
        else:
            seen.add(i)
            actual_sum+=i
    expected_sum=n*(n+1)//2
    missinng_sum=expected_sum-actual_sum
    return [missinng_sum, repeating]

print(findTwoElement())

        
       
       

[1, 2]


In [3]:
def binary_search():
    arr=[1, 2, 3, 4, 5]
    k = 4
    n=len(arr)
    for i in range(n):
        if arr[i]==k:
            return i
    return -1

print(binary_search())
    

3


In [9]:
def largest_element():
    arr= [1, 8, 7, 56, 90,44]
    arr.sort()
    for _ in arr:
        return arr[-1]

print(largest_element())

90


In [11]:
#array subset
def subset_array():
    arr1=[11, 7, 1, 13, 21, 3, 7, 3]
    arr2 = [11, 3, 7, 1, 7]
    freq={}
    for i in arr1:
        freq[i]=freq.get(i,0)+1
    for i in arr2:
        if freq.get(i,0)==0:
            return False
        freq[i]-=1
    return True
print(subset_array())

True


In [13]:
#Reverse a String
def reverse_string():
    s="Geeks"
    return s[::-1]

print(reverse_string())


skeeG


In [19]:
#anagram
def anagram():
    str1="listen"
    str2="netsil"
    if len(str1)!=len(str2):
        return False
    return sorted(str1)==sorted(str2)

print(anagram())
        

True


In [26]:
#to check isPalindrome
def isPalindrome():
    s="helleh"
    return s==s[::-1]
print(isPalindrome())

True


In [29]:
#two sum pair with given sum
def two_sum():
    arr=[1, 2, 3, 4, 5]
    target=11
    seen=set()    
    for i in arr:
        num=target-i
        if num in seen:
            return True
        seen.add(i)
    return False
    
print(two_sum())

False


In [46]:
import heapq
li = [5,12, 20, 15, 30, 40]

# Convert the list into a heap
heapq.heapify(li)
# peek_list=heapq.peek(li)  # Get the smallest element without removing it
# pop_list=heapq.heappop(li)
# print("Smallest element:", pop_list)
# pop_list1=heapq.heappop(li)
# print("Smallest element:", pop_list1)
# push_list1=heapq.  heappush(li, 23)


print("Heapified list:", li)

Heapified list: [5, 12, 20, 15, 30, 40]


In [ ]:
print(len("Hello"))
print(len([1,3,4]))

print(max(1,3,4,5))
print(min(1,2,3,4))

In [52]:
def add(a,b):
    return a+b

print(add(1,3))

4


In [54]:
#method overloading
print(5+3)
print([1,2,3] +[4,5,5])

8
[1, 2, 3, 4, 5, 5]


In [66]:
class Shape:
    def __init__(self):
        pass
    
class rectangle(Shape):
    def __init__(self,length,width):
        self.length=length
        self.width=width
        
    def area(self) :
        return self.length*self.width
    
class circle(Shape):
    def __init__(self,radius) :
        self.radius=radius
        
    def area(self):
        return 3.14*self.radius**2

shapes=[rectangle(5,10),circle(7)]
for shape in shapes:
   print(f"Area: {shape.area()}")

Area: 50
Area: 153.86


In [72]:
# Parent class
class Animal:
    def __init__(self, name):
        self.name = name  # Initialize the name attribute

    def speak(self):
        pass  # Placeholder method to be overridden by child classes

# Child class inheriting from Animal
class Dog(Animal):
    def speak(self):
        return f"{self.name} barks!"  # Override the speak method

class Cat(Animal):
    
    def speak(self):
        return f"{self.name} meows!"
# Creating an instance of Dog
dog = Dog("Buddy")
cat=Cat("kitty")
print(dog.speak())  
print(cat.speak())


Buddy barks!
kitty meows!


In [76]:
from abc import ABC, abstractmethod
class Animal:
    @abstractmethod
    def sound(self):
        pass

class Dog(Animal):
    def sound(self):
        return "bark"

animal=Dog()
animal.sound()


'bark'

In [102]:
from pydantic import BaseModel

# Define a Pydantic model
class User(BaseModel):
    id: int
    name: str
    signup_ts: str 
    friends: list[int] = []

# Create a User object
user = User(
    id=1,
    name="John Doe",
    signup_ts="2024-04-28T12:00",
    friends=[2, 3, 4]
)

# print(user)
# print(user.id)
# print(user.dict())  # Convert to dictionary


In [105]:
user_list={'id': "1", 'name': 100, 'signup_ts': '2024-04-28T12:00', 'friends': (2, 3, 4)}


In [106]:
User(**user_list)

ValidationError: 1 validation error for User
name
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type